In [1]:
# Ignore cuDDa warning messages
import warnings
import os
warnings.filterwarnings('ignore')

# Enable GPU
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# # Expands the Jupyter Notebook Output Size to fit your window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Change the working directory back to the original to keep paths the same between files
os.chdir(r'C:\Users\Maxwell\PycharmProjects\TAMU-ECEN-403-IFPTSND\ECEN_403_IFM\TAMU-ECEN-403-IFPTSND')

# warnings.filterwarnings('ignore')
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
%%javascript
    IPython.OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>

In [3]:
# Import all of my custom files
from tensorflow_models import *
from explore_entities import *
from graph_predictions import *

In [4]:
# Create our model generator object
dataset_path = './ignorable_data/data_sets/NASDAQ_Cleaned - Contains ZUMZ/'
model_path = './ignorable_data/models/[40, 40, 20]_split/'
DMJ = TF_Models(dataset_path, model_path, reload=False, perc_split=[2,2,96])

In [8]:
# Create a model
DMJ.generate_model()

GridBox(children=(Dropdown(description='Model Types:', options=('lstm', 'explicit_gcn', 'implicit_gcn', 'lstm_…

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 24, 5)]      0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 24, 16)       1408        input_3[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 24, 1)        17          lstm_1[0][0]                     
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 24, 1)        17          lstm_1[0][0]                     
_______________________________________________________________________________________

In [ ]:
# Train a model
DMJ.train(epochs=70, batch_size=200)

Epoch 1/70
1/1 [==============================] - 0s 370ms/step - loss: 0.0643 - dense_5_loss: 0.0643 - val_loss: 0.0667 - val_dense_5_loss: 0.0667
Epoch 2/70
1/1 [==============================] - 0s 252ms/step - loss: 0.0636 - dense_5_loss: 0.0636 - val_loss: 0.0660 - val_dense_5_loss: 0.0660
Epoch 3/70
1/1 [==============================] - 0s 261ms/step - loss: 0.0632 - dense_5_loss: 0.0632 - val_loss: 0.0654 - val_dense_5_loss: 0.0654
Epoch 4/70
1/1 [==============================] - 0s 263ms/step - loss: 0.0629 - dense_5_loss: 0.0629 - val_loss: 0.0648 - val_dense_5_loss: 0.0648
Epoch 5/70
1/1 [==============================] - 0s 249ms/step - loss: 0.0626 - dense_5_loss: 0.0626 - val_loss: 0.0640 - val_dense_5_loss: 0.0640
Epoch 6/70
1/1 [==============================] - 0s 244ms/step - loss: 0.0620 - dense_5_loss: 0.0620 - val_loss: 0.0630 - val_dense_5_loss: 0.0630
Epoch 7/70
1/1 [==============================] - 0s 258ms/step - loss: 0.0611 - dense_5_loss: 0.0611 - val_loss

In [ ]:
# Save the model
DMJ.save_model()
print(DMJ.model_name)

In [ ]:
# Create an object for creating models
GP = Graph_Predictions(model_path,  DMJ)

In [ ]:
# Where the prediction will be saved
prediction_path = './ignorable_data/prediction_results/'

model_name = DMJ.model_name

# The data that the model is able to see
past = DMJ.data_splits['x_val']

# The data that the model will predict over and incrementally see
future = DMJ.data_splits['x_test']

# The size of the time-window
window = DMJ.data_splits['x_val'].shape[1]

# Generate the predictions for every company on every day
# GP.generate_predictions(model_name, model_path, past, future, prediction_path, window, model_type='lstm', batch_size=881)
# model_name='04-20-2021--22--51-explicit_gcn-100Epochs-rank_loss-32HU-'
GP.generate_predictions(model_name, model_path, past, future, prediction_path, window, model_type='gcn', batch_size=881, input_Adj_matrix=DMJ.Normalized_Adjacency_Matrix)

In [ ]:
datablock_path = './ignorable_data/datablocks/[55, 25, 20]_splits/'
prediction_file = GP.most_recent_prediction_file

# Using the prediction file, execute trading strategies over the dataset
GP.generate_prediction_results_avg(5, prediction_file, prediction_path, future, datablock_path)

In [ ]:
# General Stats
GP.compare_data_blocks(datablock_path)

In [ ]:
# Advanced Interpretablility
GP.perturbate_sequential_model(DMJ.model_name, model_path, DMJ.data_splits['x_train'])